In [ ]:
# stdlib
import os

environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
environment

In [ ]:
# third party

# syft absolute
import syft as sy

In [ ]:
SERVER_PORT = "8080"
SERVER_URL = f"http://localhost:{SERVER_PORT}"

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    # reset=True,
    port=SERVER_PORT,
    n_consumers=4,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"
high_client = sy.login(
    url="http://localhost:8080", email=ADMIN_EMAIL, password=ADMIN_PW
)

In [ ]:
high_client.requests

In [ ]:
high_client.requests.get_all_approved()

In [ ]:
high_client.requests[2].code

In [ ]:
with sy.raises(sy.SyftException):
    high_client.requests[2].code()

In [ ]:
job = high_client.requests[2].code(blocking=False)

In [ ]:
job.wait()

In [ ]:
assert isinstance(job.result, sy.SyftError)

In [ ]:
high_client.requests

In [ ]:
for request in high_client.requests:
    if request.code.service_func_name.startswith("wrong_syntax_query"):
        bad_request = request
    if request.code.service_func_name.startswith("simple_query"):
        good_request = request

In [ ]:
good_job = good_request.code(blocking=False)

In [ ]:
good_job.wait()

In [ ]:
high_client.requests.get_all_approved()

In [ ]:
# stdlib
import json

# third party
from job_helpers import resolve_request

request_dict = {}

for request in high_client.requests:
    request_id, request_status = resolve_request(request)
    request_dict[str(request_id)] = str(request_status)

with open(".requests.json", "w") as fp:
    json.dump(request_dict, fp)

In [ ]:
bad_job = bad_request.code(blocking=False)

In [ ]:
bad_job

In [ ]:
bad_job.wait()

In [ ]:
retry_good_job = good_request.code(blocking=False)

In [ ]:
retry_good_job.wait()